In [ ]:
!pip install -q langgraph-checkpoint-sqlite langgraph groq langchain_anthropic langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00


In [ ]:
import sqlite3
from groq import Groq

client = Groq(api_key="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")
db_connection = sqlite3.connect("chat_memory.db")
cursor = db_connection.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS messages (
    thread_id TEXT,
    role TEXT,
    content TEXT
)
""")
db_connection.commit()

def save_message(thread_id, role, content):
    cursor.execute(
        "INSERT INTO messages (thread_id, role, content) VALUES (?, ?, ?)",
        (thread_id, role, content)
    )
    db_connection.commit()

def fetch_messages(thread_id, num_messages=5):
    cursor.execute(
        "SELECT role, content FROM messages WHERE thread_id = ? ORDER BY rowid DESC LIMIT ?",
        (thread_id, num_messages)
    )
    rows = cursor.fetchall()
    return [{"role": role, "content": content} for role, content in reversed(rows)]

def extract_assistant_reply(response):
    if hasattr(response, 'choices') and len(response.choices) > 0:
        choice = response.choices[0]
        if hasattr(choice, 'message') and hasattr(choice.message, 'content'):
            return choice.message.content
    return "Sorry, I couldn't process your request."

def print_thread_memory(thread_id):
    messages = fetch_messages(thread_id)
    print(f"\n--- Memory for Thread {thread_id} ---")
    for message in messages:
        print(f"Role: {message['role']}, Content: {message['content']}")

def chatbot(thread_id, user_input):
    save_message(thread_id, "user", user_input)
    messages = fetch_messages(thread_id)

    system_message = {"role": "system", "content": "You are a helpful chatbot. You have to reply to content in concise manner. Your name is Alex and you are a helpful chatbot"}
    messages.insert(0, system_message)

    try:
        response = client.chat.completions.create(
            model="mixtral-8x7b-32768",
            messages=messages
        )
        assistant_reply = extract_assistant_reply(response)
        save_message(thread_id, "assistant", assistant_reply)
        return assistant_reply
    except Exception as e:
        print("\n--- Error Triggered ---")
        print(str(e))
        return "There was an error processing your request. Please try again later."

def start_interactive_chat(thread_id):
    print("Starting conversation... Type 'exit' to end the chat.")

    while True:
        user_input = input("You: ")

        if user_input.lower() == "show memory":
            print_thread_memory(thread_id)
            continue

        if "remember" in user_input.lower() or "last convo" in user_input.lower():
            messages = fetch_messages(thread_id)
            if messages:
                print("\nRecalling your last conversation:\n")
                for message in messages:
                    print(f"Role: {message['role']}, Content: {message['content']}")
            else:
                print("I don't have a conversation history with you yet.")
            continue

        if user_input.lower() == "exit":
            print("Ending conversation.")
            break

        response = chatbot(thread_id, user_input)
        print(f"Bot: {response}")

thread_id = "1"
start_interactive_chat(thread_id)

Starting conversation... Type 'exit' to end the chat.
You: do you remember a poem i asked 

Recalling your last conversation:

Role: assistant, Content: Hello, Ebe! It's nice to meet you. I'm Alex, a helpful chatbot that strives to provide brief and precise responses. I'm here to answer your questions, address any concerns, or engage in a conversation on various topics. What would you like to know or talk about, Ebe? I'm all ears!
Role: user, Content: What is my name
Role: assistant, Content: Based on the information you've provided, your name is Ebe. I'm here to assist you and make sure your experience is as smooth as possible. If you have any questions about a specific topic, feel free to ask, and I'll do my best to provide clear and concise answers.
Role: user, Content: give me a poem about samosa
Role: assistant, Content: In the realm of savory, there lies a gem,
Samosa, an Indian joy, shaped like a stem.
A triangular pastry of golden delight,
Filled with spiced vegetables, such sp

KeyboardInterrupt: Interrupted by user